# Jobcloud data cleaning

In [1]:
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 200)

In [2]:
df = pd.read_csv('Data/jobcloud_propulsion_dataset.csv', sep=';', parse_dates=['start_dt', 'end_dt'])
df.head()

,id,start_dt,end_dt,1d_view_cnt,10d_view_cnt,30d_view_cnt,package_id,industry_name,job_location,job_postal_code,contract_pct_from,contract_pct_to,title
0,8501672,2018-10-25,2018-11-26,0.2372,0.4565,0.7327,B,Industrie diverse,Espace Mittelland,NaN,100.0,100.0,"Softwarearchitekt / Projektmanager (m/w) - All-in-One Datenmanagement in Design, Produktion und Qual"
1,8501682,2018-10-25,2018-11-26,0.2883,0.5826,1.0991,B,Maschinen-/Anlagenbau,Region Biel,NaN,100.0,100.0,Prozessingenieur Lasertechnologie - Industrialisierung innovativer Fertigungstechnologien
2,8570830,2018-11-26,2018-12-28,0.1982,0.8468,1.1532,B,Industrie diverse,Espace Mittelland,NaN,100.0,100.0,"Softwarearchitekt / Projektmanager (m/w) - All-in-One Datenmanagement in Design, Produktion und Qual"
3,8649301,2019-01-08,2019-02-08,0.2883,0.7177,1.4835,B,Maschinen-/Anlagenbau,Espace Mittelland,NaN,100.0,100.0,Projektleiter (m/w) - Werkzeug- oder Maschinenbau
4,8730602,2019-02-12,2019-02-21,0.3574,0.7297,0.7297,B,Industrie diverse,Region Biel,NaN,100.0,100.0,Fachverantwortlichen Metrologie - Produkteentwicklung und -validierung


## Drop rows with missing values

In [3]:
# Drop rows with nan
df = df.dropna(subset=['1d_view_cnt', '10d_view_cnt', '30d_view_cnt', 'industry_name', 'job_location'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101735 entries, 0 to 103117
Data columns (total 13 columns):
id                   101735 non-null int64
start_dt             101735 non-null datetime64[ns]
end_dt               101735 non-null datetime64[ns]
1d_view_cnt          101735 non-null float64
10d_view_cnt         101735 non-null float64
30d_view_cnt         101735 non-null float64
package_id           101735 non-null object
industry_name        101735 non-null object
job_location         101735 non-null object
job_postal_code      52156 non-null object
contract_pct_from    101735 non-null float64
contract_pct_to      101735 non-null float64
title                101735 non-null object
dtypes: datetime64[ns](2), float64(5), int64(1), object(5)
memory usage: 10.9+ MB


## Remove html tags and replace html entities by unicode

In [4]:
# Strip html tags, see https://stackoverflow.com/a/4869782/94289
import re

df['title'] = df['title'].str.replace('<[^<]+?>', ' ') # Replace html tags by blank 
# df[df['title'].str.contains('<br/>')]

In [5]:
import html
# Replace html entities by unicode characters, e.g. &amp; => &
df['title'] = df['title'].apply(html.unescape)
df['industry_name'] = df['industry_name'].apply(html.unescape)
df.loc[40, 'title']

'M&A Spezialist (m/w), 100%'

## Clean titles
Regex online parser: https://regex101.com/

In [6]:
df['title_clean'] = df['title']

# Remove appended female form, e.g. FilialleiterIn => Filialleiter
df['title_clean'] = df['title_clean'].str.replace(r'\BIn\b', '')

# Convert all to lowercase
df['title_clean'] = df['title_clean'].str.lower()


# Match a single character not present in the list below [^\w&]
#  - \w match any word character in any script (equal to [\p{L}\p{N}_])
#  - & matches the character & literally (case sensitive)
df['title_clean'] = df['title_clean'].str.replace(r'[^\w&]', ' ')

# Remove numbers
df['title_clean'] = df['title_clean'].str.replace(r'[0-9]', '') 

# Remove specific words
df['title_clean'] = df['title_clean'].str.replace(r'(\bm\b|\bw\b|\bf\b|\br\b|\bin\b|\binnen\b|\bmw\b|\bdach\b|\bd\b|\be\b|\bi\b)', '')
# Special case M&A Spezialist: m is removed so &a is replaced by m&a
df['title_clean'] = df['title_clean'].str.replace(r'&a\b', 'm&a')

# Remove qualifications
df['title_clean'] = df['title_clean'].str.replace(r'(\bdipl\b|\bfachausweis\b|\babschluss\b|diplom|phd|msc|\buni\b|\bfh\b|\bfh\b|\beth\b|\btu\b)', '')

# Replace two or more consecutive spaces by only one space
df['title_clean'] = df['title_clean'].str.replace(r'[ ]{2,}', ' ')

# Remove spaces at the start and end
df['title_clean'] = df['title_clean'].str.strip()

df.loc[:, ['title', 'title_clean']].head(5)


,title,title_clean
0,"Softwarearchitekt / Projektmanager (m/w) - All-in-One Datenmanagement in Design, Produktion und Qual",softwarearchitekt projektmanager all one datenmanagement design produktion und qual
1,Prozessingenieur Lasertechnologie - Industrialisierung innovativer Fertigungstechnologien,prozessingenieur lasertechnologie industrialisierung innovativer fertigungstechnologien
2,"Softwarearchitekt / Projektmanager (m/w) - All-in-One Datenmanagement in Design, Produktion und Qual",softwarearchitekt projektmanager all one datenmanagement design produktion und qual
3,Projektleiter (m/w) - Werkzeug- oder Maschinenbau,projektleiter werkzeug oder maschinenbau
4,Fachverantwortlichen Metrologie - Produkteentwicklung und -validierung,fachverantwortlichen metrologie produkteentwicklung und validierung


## Fix column types

In [7]:
# Since there are no more missing values, contract_pct float columns can be converted to int
df['contract_pct_from'] = df['contract_pct_from'].astype(int)
df['contract_pct_to'] = df['contract_pct_to'].astype(int)

## Save cleaned data

In [8]:
# Save cleaned dataframe in the same format as the raw data, so changes are easily visible in a diff.
#import csv
#df.to_csv('Data/jobcloud_clean.csv', sep=';',  quoting=csv.QUOTE_NONNUMERIC, index=False, float_format='%.4f')
df.to_csv('Data/jobcloud_clean_v2.csv', sep=';', index=False)